# EDA

### 라이브러리(import)

In [2]:
import pandas as pd

### 파일 불러오기(read_csv)

In [3]:
train = pd.read_csv(".data/train.csv")
test = pd.read_csv(".data/test.csv")

### 행열갯수 확인(shape)

In [4]:
print(train.shape)
print(test.shape)

(1459, 11)
(715, 10)


### 데이터 확인(head())

In [5]:
train.head(3)

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
0,3,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,49.0
1,6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159.0
2,7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26.0


### 결측치 확인하기(isnull())

`` 결측치(Missing Value) `` 는 데이터에 값이 없는 것을 의미한다. 줄여서 `` NA `` 라고 표기한다. pandas에선 `` NaN `` 으로 표현한다. pandas의 `` isnull() `` 함수는 값을 확인하여 결측치면 True, 아니면 False를 반환한다.

In [6]:
# 각 데이터의 결측지 여부 확인
print(train.isnull())

         id   hour  hour_bef_temperature  hour_bef_precipitation  \
0     False  False                 False                   False   
1     False  False                 False                   False   
2     False  False                 False                   False   
3     False  False                 False                   False   
4     False  False                 False                   False   
...     ...    ...                   ...                     ...   
1454  False  False                 False                   False   
1455  False  False                 False                   False   
1456  False  False                 False                   False   
1457  False  False                 False                   False   
1458  False  False                 False                   False   

      hour_bef_windspeed  hour_bef_humidity  hour_bef_visibility  \
0                  False              False                False   
1                  False              False    

In [8]:
# 데이터의 열별 결측치 개수를 확인
print(train.isnull().sum())

id                          0
hour                        0
hour_bef_temperature        2
hour_bef_precipitation      2
hour_bef_windspeed          9
hour_bef_humidity           2
hour_bef_visibility         2
hour_bef_ozone             76
hour_bef_pm10              90
hour_bef_pm2.5            117
count                       0
dtype: int64


# 전처리

### 파이썬 데이터 기본 정보 확인(info())

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      1459 non-null   int64  
 1   hour                    1459 non-null   int64  
 2   hour_bef_temperature    1457 non-null   float64
 3   hour_bef_precipitation  1457 non-null   float64
 4   hour_bef_windspeed      1450 non-null   float64
 5   hour_bef_humidity       1457 non-null   float64
 6   hour_bef_visibility     1457 non-null   float64
 7   hour_bef_ozone          1383 non-null   float64
 8   hour_bef_pm10           1369 non-null   float64
 9   hour_bef_pm2.5          1342 non-null   float64
 10  count                   1459 non-null   float64
dtypes: float64(9), int64(2)
memory usage: 125.5 KB


In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 715 entries, 0 to 714
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      715 non-null    int64  
 1   hour                    715 non-null    int64  
 2   hour_bef_temperature    714 non-null    float64
 3   hour_bef_precipitation  714 non-null    float64
 4   hour_bef_windspeed      714 non-null    float64
 5   hour_bef_humidity       714 non-null    float64
 6   hour_bef_visibility     714 non-null    float64
 7   hour_bef_ozone          680 non-null    float64
 8   hour_bef_pm10           678 non-null    float64
 9   hour_bef_pm2.5          679 non-null    float64
dtypes: float64(8), int64(2)
memory usage: 56.0 KB


### 결측치 삭제, 대체하기(dropna, fillna)

In [16]:
train = train.dropna()
test = test.fillna(0)

# 모델링

### 파이썬 scikit-learn

In [17]:
import sklearn
from sklearn.tree import DecisionTreeClassifier

### 의사결정나무

`` 의사결정나무 `` 는 의사 결정 규칙과 그 결과들을 트리 구조로 도식화한 일종의 의사 결정 지원 도구이다. 말이 어렵지 결국 질문들을 통해 답을 찾는 스무고개와 같다.

![의사결정나무](https://camo.githubusercontent.com/abd13c8a3da89c443c27d2f926fcde9a6d53dff03724ab65c246405f74aa00c9/68747470733a2f2f626c6f672e6b616b616f63646e2e6e65742f646e2f776c4831752f62747177575a493958656e2f6b464a446a4753464a415078687961744333586873302f696d672e706e67)

예를 들어, 위에서 살펴본 데이터에서 하나의 feature에 대해 특정 값을 정하면 이를 기준으로 모든 행들을 두 개의 node로 분류할 수 있다. 대표적인 의사결정나무인 `` CART 의사결정 나무 `` 는 이진분할을 사용한다. 이후 내용은 CART 의사결정 나무를 기반으로 한다.

하나의 feature로 두 개의 노드로 나뉘었다면 다시 새로운 feature로 값을 정하고 분류한다. 이를 반복하여 feature에 따라 data들이 분류되는 것이 의사결정 나무의 원리이다.

특정한 값을 정하는 의사결정 나무의 대원칙은 `` 한쪽 방향으로 쏠리도록 `` 이다. 이는 한쪽 방향으로 쏠리게 하는 특정 값을 찾겠다는 것이며 이는 불순도를 통해 찾아낸다.

우리가 위에서 불러온 모듈 역시 의사결정 나무이다.

### 모델선언(의사결정나무)

In [24]:
from sklearn.tree import DecisionTreeRegressor

### 모델 훈련(의사결정나무)

모델을 선언하면 `` fit(X, Y) `` 함수로 모델을 훈련시킬 수 있다. 이때 X, Y 데이터의 용도는 다음과 같다.

- X: 예측에 사용되는 변수(drop을 통해 예측할 feature를 제거)
- Y: 예측결과 변수(예측할 feature만)

In [25]:
X_train = train.drop(['count'], axis=1)
Y_train = train['count']

In [26]:
model = DecisionTreeRegressor()
model.fit(X_train, Y_train)

DecisionTreeRegressor()

### 테스트 예측

In [27]:
pred = model.predict(test)
pred[:5]

array([106., 164., 114.,  49.,  93.])

### 제출 파일 생성(to_csv())

In [29]:
sub = pd.read_csv(".data/submission.csv")
sub['count'] = pred
sub.to_csv(".data/submission.csv", index=False)